new model
main


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from google.colab import drive
import zipfile
import os


In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Section 3: Extract Images from Zip File and Create DataFrame
zip_file_path = '/content/drive/MyDrive/images/PCOS.zip'
extracted_folder = '/content/PCOS/PCOS/PCOS/PCOS/PCOS'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Verify directory structure and count the number of images
for class_name in ['Healthy', 'Unhealthy']:
    class_folder = os.path.join(extracted_folder, class_name)
    num_images = len([f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    print(f"Number of images in {class_name} class: {num_images}")

# Create a DataFrame with file paths and labels
df = pd.DataFrame(columns=['imagePath', 'Healthy'])
for class_name in ['Healthy', 'Unhealthy']:
    class_folder = os.path.join(extracted_folder, class_name)
    filepaths = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    labels = [class_name] * len(filepaths)
    class_df = pd.DataFrame({'imagePath': filepaths, 'Healthy': labels})
    df = df.append(class_df, ignore_index=True)

Number of images in Healthy class: 903
Number of images in Unhealthy class: 2297


<ipython-input-4-1924a046a838>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(class_df, ignore_index=True)
<ipython-input-4-1924a046a838>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(class_df, ignore_index=True)


In [ ]:
# Section 4: Shuffle DataFrame and Resize Images
df = df.sample(frac=1).reset_index(drop=True)
img_size = (128, 128)


In [ ]:
# Section 5: Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Section 6: Create Data Generators
batch_size = 32
train_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='imagePath',
    y_col='Healthy',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)


Found 3200 validated image filenames belonging to 2 classes.


In [ ]:
# Section 7: Build Improved CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Section 8: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Section 9: Learning Rate Scheduler
def lr_schedule(epoch):
    return 0.001 * 0.9 ** epoch

lr_scheduler = LearningRateScheduler(lr_schedule)


In [ ]:
# Section 10: Load Model Checkpoint if Available
last_epoch =350
model_checkpoint_path = '/content/drive/MyDrive/healthy_model1.h5'
if os.path.exists(model_checkpoint_path):
    model.load_weights(model_checkpoint_path)

In [ ]:
# Section 11: Train the Model
epochs =500
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    initial_epoch=last_epoch,
    callbacks=[lr_scheduler],
)

# Section 12: Save the Trained Model
model.save('/content/drive/MyDrive/healthy_model1.h5')

Epoch 351/500
100/100 [==============================] - 28s 193ms/step - loss: 0.4337 - accuracy: 0.7800 - lr: 9.6578e-20
Epoch 352/500
100/100 [==============================] - 19s 191ms/step - loss: 0.4286 - accuracy: 0.7825 - lr: 8.6920e-20
Epoch 353/500
100/100 [==============================] - 19s 191ms/step - loss: 0.4325 - accuracy: 0.7763 - lr: 7.8228e-20
Epoch 354/500
100/100 [==============================] - 20s 200ms/step - loss: 0.4295 - accuracy: 0.7791 - lr: 7.0405e-20
Epoch 355/500
100/100 [==============================] - 20s 194ms/step - loss: 0.4337 - accuracy: 0.7791 - lr: 6.3365e-20
Epoch 356/500
100/100 [==============================] - 21s 211ms/step - loss: 0.4345 - accuracy: 0.7809 - lr: 5.7028e-20
Epoch 357/500
100/100 [==============================] - 20s 195ms/step - loss: 0.4352 - accuracy: 0.7772 - lr: 5.1326e-20
Epoch 358/500
100/100 [==============================] - 19s 194ms/step - loss: 0.4289 - accuracy: 0.7784 - lr: 4.6193e-20
Epoch 359/500
10

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import zipfile
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import shutil

zip_path = '/content/drive/MyDrive/images/PCOSGen-test.zip'
extract_path = '/content/test_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


model_path = '/content/drive/MyDrive/healthy_model1.h5'
model = load_model(model_path)


output_path = '/content/drive/MyDrive'
healthy_folder = os.path.join(output_path, 'healthy')
unhealthy_folder = os.path.join(output_path, 'unhealthy')

os.makedirs(healthy_folder, exist_ok=True)
os.makedirs(unhealthy_folder, exist_ok=True)


predictions_df = pd.DataFrame(columns=['Image', 'Prediction', 'Status'])


for subdir, dirs, files in os.walk(os.path.join(extract_path, 'images')):
    for image_file in files:
        image_path = os.path.join(subdir, image_file)

      
        img = image.load_img(image_path, target_size=(128, 128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

    
        predictions = model.predict(img_array)


        status = 'Healthy' if predictions[0] < 0.5 else 'Unhealthy'

       
        predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)

        
        output_folder = healthy_folder if status == 'Healthy' else unhealthy_folder
        output_file_path = os.path.join(output_folder, image_file)
        shutil.move(image_path, output_file_path)

# Save the Excel sheet to Google Drive
excel_output_path = '/content/drive/MyDrive/predictions_with_status2.xlsx'
predictions_df.to_excel(excel_output_path, index=False)

print("Prediction and organization completed.")


1/1 [==============================] - 0s 27ms/step


<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)


1/1 [==============================] - 0s 93ms/step


<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)


1/1 [==============================] - 0s 67ms/step


...

<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)


1/1 [==============================] - 0s 19ms/step


<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)


1/1 [==============================] - 0s 17ms/step
Prediction and organization completed.


<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)
<ipython-input-5-536ded4648ef>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Image': image_file, 'Prediction': predictions[0][0], 'Status': status}, ignore_index=True)


In [ ]:
import pandas as pd
from sklearn.metrics import average_precision_score, accuracy_score, f1_score

# Load the predictions file
predictions_df = pd.read_excel('/content/drive/MyDrive/predictions_with_status2.xlsx')  # Replace with your predictions file path


y_true = predictions_df['Status'].apply(lambda x: 1 if x == 'Unhealthy' else 0).values
y_pred_probabilities = predictions_df['Prediction'].values


thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Calculate mAP, accuracy, and F1 score for each threshold
for threshold in thresholds:
   
    y_pred_binary = (y_pred_probabilities >= threshold).astype(int)


    map_score = average_precision_score(y_true, y_pred_binary)

 
    accuracy = accuracy_score(y_true, y_pred_binary)


    f1 = f1_score(y_true, y_pred_binary)

    print(f'Threshold: {threshold}, Mean Average Precision (mAP): {map_score}, Accuracy: {accuracy}, F1 Score: {f1}')


Threshold: 0.1, Mean Average Precision (mAP): 0.9748784440842788, Accuracy: 0.978882833787466, F1 Score: 0.9872794419368076
Threshold: 0.2, Mean Average Precision (mAP): 0.9828431372549019, Accuracy: 0.9856948228882834, F1 Score: 0.9913473423980221
Threshold: 0.3, Mean Average Precision (mAP): 0.9868744872846595, Accuracy: 0.989100817438692, F1 Score: 0.9933938893476466
Threshold: 0.4, Mean Average Precision (mAP): 0.9893092105263158, Accuracy: 0.9911444141689373, F1 Score: 0.9946258784621744
Threshold: 0.5, Mean Average Precision (mAP): 1.0, Accuracy: 1.0, F1 Score: 1.0
Threshold: 0.6, Mean Average Precision (mAP): 0.9941478048747342, Accuracy: 0.973433242506812, F1 Score: 0.9835234474017743
Threshold: 0.7, Mean Average Precision (mAP): 0.9704389118031442, Accuracy: 0.8658038147138964, F1 Score: 0.9108193752829334
Threshold: 0.8, Mean Average Precision (mAP): 0.9599349718347183, Accuracy: 0.8181198910081744, F1 Score: 0.8751753155680224
Threshold: 0.9, Mean Average Precision (mAP): 0.